# FINM 25000 - Quantitative Portfolio Management and Algorithmic Trading

## Mid Term Solutions

In [5]:
# Start by importing the packages and datasets

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def performanceMetrics(returns,annualization=1, quantile=.05):
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Mean'] = returns.mean() * annualization
    metrics['Vol'] = returns.std() * np.sqrt(annualization)
    metrics['Sharpe'] = (returns.mean() / returns.std()) * np.sqrt(annualization)

    metrics['Min'] = returns.min()
    metrics['Max'] = returns.max()
    return metrics

def tailMetrics(returns, quantile=.05, relative=False, mdd=True):
    
    #Maximum Drawdown
    def maximumDrawdown(returns):
        cum_returns = (1 + returns).cumprod()
        rolling_max = cum_returns.cummax()
        drawdown = (cum_returns - rolling_max) / rolling_max

        max_drawdown = drawdown.min()
        end_date = drawdown.idxmin()
        summary = pd.DataFrame({'Max Drawdown': max_drawdown, 'Bottom': end_date})

        for col in drawdown:
            summary.loc[col,'Peak'] = (rolling_max.loc[:end_date[col],col]).idxmax()
            recovery = (drawdown.loc[end_date[col]:,col])
            try:
                summary.loc[col,'Recover'] = pd.to_datetime(recovery[recovery >= 0].index[0])
            except:
                summary.loc[col,'Recover'] = pd.to_datetime(None)

            summary['Peak'] = pd.to_datetime(summary['Peak'])
            try:
                summary['Duration (to Recover)'] = (summary['Recover'] - summary['Peak'])
            except:
                summary['Duration (to Recover)'] = None

            summary = summary[['Max Drawdown','Peak','Bottom','Recover','Duration (to Recover)']]

        return summary  
    
    metrics = pd.DataFrame(index=returns.columns)
    metrics['Skewness'] = returns.skew()
    metrics['Kurtosis'] = returns.kurtosis()

    VaR = returns.quantile(quantile)
    CVaR = (returns[returns < returns.quantile(quantile)]).mean()

    if relative:
        VaR = (VaR - returns.mean())/returns.std()
        CVaR = (CVaR - returns.mean())/returns.std()

    metrics[f'VaR ({quantile})'] = VaR
    metrics[f'CVaR ({quantile})'] = CVaR

    if mdd:
        mdd_stats = maximumDrawdown(returns)
        metrics = metrics.join(mdd_stats)

        if relative:
            metrics['Max Drawdown'] = (metrics['Max Drawdown'] - returns.mean())/returns.std()

    return metrics


def get_ols_metrics(regressors, targets, annualization=1, ignorenan=True):
    # ensure regressors and targets are pandas dataframes, as expected
    if not isinstance(regressors, pd.DataFrame):
        regressors = regressors.to_frame()
    if not isinstance(targets, pd.DataFrame):
        targets = targets.to_frame()

    # align the targets and regressors on the same dates
    df_aligned = targets.join(regressors, how='inner', lsuffix='y ')
    Y = df_aligned[targets.columns]
    Xset = df_aligned[regressors.columns]

    reg = pd.DataFrame(index=targets.columns)
    for col in Y.columns:
        y = Y[col]
        
        if ignorenan:
            # ensure we use only non-NaN dates
            alldata = Xset.join(y,lsuffix='X')
            mask = alldata.notnull().all(axis=1)
            y = y[mask]
            X = Xset[mask]
        else:
            X = Xset

        model = LinearRegression().fit(X, y)
        reg.loc[col, 'alpha'] = model.intercept_ * annualization
        reg.loc[col, regressors.columns] = model.coef_
        reg.loc[col, 'r-squared'] = model.score(X, y)

        # sklearn does not return the residuals, so we need to build them
        yfit = model.predict(X)
        residuals = y - yfit

        # Treynor Ratio is only defined for univariate regression
        if Xset.shape[1] == 1:
            reg.loc[col,'Treynor Ratio'] = (y.mean() / model.coef_) * annualization

        
        # if intercept =0, numerical roundoff will nonetheless show nonzero Info Ratio
        num_roundoff = 1e-12
        if np.abs(model.intercept_) < num_roundoff:
            reg.loc[col, 'Info Ratio'] = None
        else:
            reg.loc[col, 'Info Ratio'] = (model.intercept_ / residuals.std()) * np.sqrt(annualization)

    return reg

def tangency_portfolio(data):
    mu = data.mean()
    sigma = np.linalg.inv(data.cov())
    one_vector = np.ones(len(data.columns))
    return sigma @ mu / (one_vector @ sigma @ mu)

def tangency_portfolio_allocation(data, target_return = 0.01):
    mu = data.mean()
    sigma = np.linalg.inv(data.cov())
    one_vector = np.ones(len(data.columns))
    tan_wts = tangency_portfolio(data)
    return ((one_vector @ sigma @ mu) / (mu @ sigma @ mu)) * target_return 

In [6]:
factors = pd.read_excel('midterm_A_data.xlsx', 0).set_index('Date')
assets = pd.read_excel('midterm_A_data.xlsx', 1).set_index('Date')
risk_free = pd.read_excel('midterm_A_data.xlsx', 2).set_index('Date')

# 1 Return Analysis

1. 

    **(a) For each of the 10assets, report the following annualized excess return statistics:**
        • mean
        • volatility
        • Sharpe ratio
    **(b) Which asset has the**
        • highest mean return?
        • highest Sharpe ratio?

In [7]:
perform_metrics = performanceMetrics(assets, annualization=12)[['Mean', 'Vol', 'Sharpe']]
perform_metrics = perform_metrics.sort_values('Sharpe', ascending = False)
display(perform_metrics.style.format('{:,.2%}'))
print('Asset with Max Mean Return: {}'.format(perform_metrics.Mean.idxmax()))
print('Asset with Max Sharpe: {}'.format(perform_metrics.iloc[0].name))

,Mean,Vol,Sharpe
NoDur,8.66%,12.49%,69.28%
Utils,9.16%,14.53%,63.03%
Hlth,7.86%,14.23%,55.28%
Shops,8.33%,15.81%,52.72%
Manuf,8.71%,17.37%,50.16%
Enrgy,9.98%,24.60%,40.56%
Durbl,10.73%,30.07%,35.69%
Other,6.56%,18.44%,35.58%
HiTec,7.24%,23.77%,30.45%
Telcm,2.35%,18.19%,12.91%


Asset with Max Mean Return: Durbl
Asset with Max Sharpe: NoDur


**2. For each of the 10 assets, report the following statistics (no annualization needed).**
        
        • VaR (0.05). That is to say, the 5th quantile of returns.
        
        • CVaR (0.05). That is to say, the average of the returns less than the 5th quantile.
        
        • Maximum Drawdown. Though we usually calculate maximum drawdown on total returns,
        keep things simple and just continue to use the excess returns we’re already using in all
        the other problems.

In [8]:
tail_metrics = tailMetrics(assets)[['VaR (0.05)', 'CVaR (0.05)', 'Max Drawdown']]
display(tail_metrics.style.format('{:,.2%}'))

,VaR (0.05),CVaR (0.05),Max Drawdown
NoDur,-5.73%,-7.95%,-34.99%
Durbl,-11.70%,-16.07%,-76.69%
Manuf,-8.50%,-11.57%,-54.91%
Enrgy,-10.71%,-14.72%,-66.39%
HiTec,-12.12%,-15.96%,-81.35%
Telcm,-9.53%,-12.16%,-77.39%
Shops,-7.48%,-9.67%,-40.91%
Hlth,-7.18%,-8.85%,-42.58%
Utils,-6.46%,-9.72%,-39.23%
Other,-8.41%,-12.56%,-69.57%


**3. Which pair of assets has the highest correlation? And the smallest correlation?**

In [9]:
cor_table = assets.corr()
cor_table[cor_table == 1] = None
cor_table = cor_table.unstack().dropna()
max_cor = cor_table.max()
min_cor = cor_table.min()

print('Highest correlation is between: '+ str(cor_table[cor_table==max_cor].index[0]))
print('Lowest correlation is between: '+ str(cor_table[cor_table==min_cor].index[0]))

Highest correlation is between: ('Manuf', 'Other')
Lowest correlation is between: ('HiTec', 'Utils')


# 2 Mean-Variance Optimization

**1. Calculate the weights of the tangency portfolio formed from the 10 assets.**

In [10]:
#Tangency Portfolio Weights Vector
tangency_portfolio_weights = tangency_portfolio(assets)
pd.DataFrame(tangency_portfolio_weights, index = assets.columns, columns = ['Weights for the Tangency Portfolio']).sort_values('Weights for the Tangency Portfolio', ascending = False).style.format('{:,.2%}')

,Weights for the Tangency Portfolio
NoDur,74.75%
Shops,42.92%
Utils,29.65%
Hlth,28.27%
HiTec,16.23%
Enrgy,10.91%
Manuf,6.62%
Durbl,1.60%
Other,-49.59%
Telcm,-61.36%


**2. (a) What are the weights of the optimal portfolio, $w^*$, with a targeted mean excess return of 0.01 per month?**
    **(b) Is the optimal portfolio, $w^∗$, invested in the risk-free rate?**

In [11]:
# Allocation to tangency portfolio
portfolio_weights = tangency_portfolio_weights * tangency_portfolio_allocation(assets, )
pd.DataFrame(portfolio_weights, index = assets.columns, columns = ['Weights for the Target Portfolio']).sort_values('Weights for the Target Portfolio', ascending = False).style.format('{:,.2%}')

,Weights for the Target Portfolio
NoDur,67.44%
Shops,38.72%
Utils,26.74%
Hlth,25.51%
HiTec,14.64%
Enrgy,9.84%
Manuf,5.97%
Durbl,1.45%
Other,-44.74%
Telcm,-55.35%


In [12]:
invst_rf = 1 - sum(portfolio_weights)
print('Yes, the optimal portfolio has {:,.2%} weight in the risk free rate.'.format(invst_rf))

Yes, the optimal portfolio has 9.79% weight in the risk free rate.


**3. Report the mean, volatility, and Sharpe ratio for the optimized portfolio, $w∗$, (calculated in the previous question.) Annualize the statistics.**

In [13]:
performanceMetrics(pd.DataFrame(assets@portfolio_weights, columns=['Optimal Portfolio']), annualization=12)\
.style.format('{:,.2%}')

,Mean,Vol,Sharpe,Min,Max
Optimal Portfolio,12.00%,12.24%,98.01%,-10.19%,13.61%


**4. Suppose an endowment is optimizing the multi-asset-class ETFs we used in our case study of Homework 1. Briefly describe one thing that should be considered in deciding whether to make Bitcoin an (additional) asset class for the endowment’s allocation.**

The endowment should consider Bitcoin's return correlation with other assets/asset classes before including it in the allocation. Lower correlation to other assets is preferred to improve portfolio diversification, thereby increase portfolio Sharpe Ratio.

**5. In Homework 1, we tried estimating the mean-variance solution using data from 2009-2020 and testing it out of sample in 2021-2022.
We found the out-of-sample performance of the MV solution was worse than an equally weighted
portfolio.
Explain from a technical perspective why MV failed out of sample.**

MV Optimization fails out-of-sample for two reasons:

1. ***Imprecise estimation of covariance matrix:*** The covariance matrix is poorly estimated in the case of large number of assets or less amount of historical data. Inverting the covariance matrix makes the estimation even more fragile. Inverting a matrix with high correlations increases the condition number further adding to the instability. Due to these, our estimates of covariances will likely not hold out-of-sample.
2. ***High senstivity to changes in mean return:*** MV optimizer is highly sensitive to small changes in the estimated mean returns of the security pool. Large swings in portfolio weights are required to maintain the optimal portfolio even with small changes in mean returns. Due to this, MV optimizer does not perform well on out-of-sample data   

# 3 Pricing

**1. Test a 3-factor pricing model on the 10assets. All this data is already given in excess returns, so no further adjustment is needed. Report the:**

    - annualized alphas
    - annualized Information ratios
    - r-squared statistics

In [14]:
pricing_reg = get_ols_metrics(factors, assets, annualization=12)
display(pricing_reg.style.format('{:,.2%}'))

,alpha,MKT,HML,RMW,r-squared,Info Ratio
NoDur,1.78%,66.46%,12.56%,40.79%,63.82%,23.74%
Durbl,-1.91%,158.66%,20.04%,33.18%,63.84%,-10.55%
Manuf,-0.38%,107.91%,18.53%,28.88%,88.13%,-6.28%
Enrgy,0.85%,99.10%,65.11%,13.80%,47.57%,4.77%
HiTec,3.07%,121.60%,-48.38%,-40.91%,89.18%,39.26%
Telcm,-3.57%,93.56%,2.03%,-3.38%,66.51%,-33.93%
Shops,0.42%,94.06%,-5.27%,37.21%,76.10%,5.40%
Hlth,3.36%,66.42%,-8.60%,8.81%,51.36%,33.91%
Utils,3.73%,51.80%,15.34%,29.96%,30.03%,30.72%
Other,-2.11%,107.62%,50.95%,3.09%,92.01%,-40.39%


**2. Which asset does the pricing model fit best?**

In [15]:
print('Pricing model fits {} asset class the best because it has the lowest absolute alpha.'\
      .format(pricing_reg.alpha.abs().idxmin()))

Pricing model fits Manuf asset class the best because it has the lowest absolute alpha.


**3. (5pts) Instead of the 3-factor model above, suppose the CAPM is true and fits perfectly in our
sample. For n assets, what do we know about their...**

    • time-series r-squared metrics?
    • Treynor Ratios?
    • Information Ratios?
    
    
**time-series r-squared metrics:** Nothing. Time-series R-Squared values can be high or low even if the CAPM is perfect, because CAPM doesn't claim to explain the difference in returns of a security at different times. CAPM says that long term expected excess return of a security if a linear function of it's beta to the market premium.

**Treynor Ratio:** If CAPM were true, Treynor Ratio $\frac{\mathbb{E}[\tilde{r}]}{\beta}$ for every security would be equal to the expected market premium. 

**Information Ratio:** If CAPM were true, there would be no excess expected return against the market premium. Hence alpha would be zero and Info Ratio would be zero.

**4. Suppose the CAPM is true and fits perfectly in our sample, yet we estimated the 3-factor model as above. Would the betas on the extra regressors (HML and RMW) be zero?**

Even if CAPM is true and fits perfectly in the sample, it is not necessary that the time-series betas on other factors would be zero. That's because these factors can still help explain the variation in a security returns time series so that time-series model error terms are even smaller.

# 4 Forecasting

**1. Forecast the market return, MKT, using the lagged risk-free rate as a signal. Report the beta and r-squared from the regression.**

In [16]:
lag_risk_free = risk_free.shift().dropna()
mkt = factors[['MKT']].iloc[1:]
model = LinearRegression().fit(lag_risk_free, mkt)



model_params = pd.DataFrame({'Beta': model.coef_[0], 'R-Squared' : model.score(lag_risk_free, mkt)}, index = ['Forecast'])
display(model_params.style.format({'Beta':'{:,.2f}', 'R-Squared' : '{:,.2%}'}))

,Beta,R-Squared
Forecast,-4.63,2.31%


In [17]:
lag_risk_free

,RF
Date,
2000-02-29,0.00
2000-03-31,0.00
2000-04-30,0.00
2000-05-31,0.00
2000-06-30,0.01
...,...
2022-01-31,0.00
2022-02-28,0.00
2022-03-31,0.00


In [18]:
mkt

,MKT
Date,
2000-02-29,0.02
2000-03-31,0.05
2000-04-30,-0.06
2000-05-31,-0.04
2000-06-30,0.05
...,...
2022-01-31,-0.06
2022-02-28,-0.02
2022-03-31,0.03


**2. Calculate the fitted values.**

$$\hat{y_t} = \alpha + \beta x_t$$

**Use them to calculate the weights:**
$$w_t = 100\hat{y_t} = 100(\alpha + \beta x_t)$$

**Report the final value in the timeseries of $w_t$**

In [19]:
predict_mkt = model.predict(lag_risk_free)
wt = 100*predict_mkt
print(wt[len(wt)-1])

[1.12435602]


**3. Use the weights to calculate the strategy return:**
$$r^x_{t+1} = w_t r^{\text{MKT}}_{t+1}$$

Report the **annualized** mean, volatility and Sharpe ratio of the strategy.

In [20]:
rt = wt*mkt
performanceMetrics(rt, 12).style.format('{:,.2%}')

,Mean,Vol,Sharpe,Min,Max
MKT,9.41%,13.67%,68.85%,-11.36%,15.77%


**4. Estimate the factor decomposition of the strategy versus the MKT factor:**

$$r^x_t = \alpha + \beta r^{\text{MKT}}_t + \epsilon_t$$

**Report**
* the annualized alpha and Info Ratio. 
* beta
* r-squared.

In [21]:
strat_model = LinearRegression().fit(mkt,rt)
rsq = strat_model.score(mkt,rt)
alpha = strat_model.intercept_[0]
beta = strat_model.coef_[0]
resid = rt - strat_model.predict(mkt)
info_ratio = alpha/resid.std()

decomp = pd.DataFrame({'alpha': alpha*12, 'Info Ratio': info_ratio*np.sqrt(12), 'MKT': beta, 'r-squared': rsq})
display(decomp.style.format('{:,.2%}'))

,alpha,Info Ratio,MKT,r-squared
MKT,5.94%,54.13%,51.86%,35.54%


**5. Suppose you wanted to hedge the timing strategy against movements in MKT. Based on the previous calculation, explain how to set up this hedge.**

To hedge the strategy above, we would *short* 0.5186 units of the MKT for every unit of investment in the strategy. The optimal hedge ratio of 0.5186 is given by the beta of the regression